# Практическая работа 1. Источники данных. Извлечение данных



ФИО:  Овсепян Милена Виталиевна

Группа: АДЭУ-201

Используем данные из открытого источника для `CSV`, `Parquet` и `API`, а также вручную подготовим данные для баз данных и HTML  из открытых данных (доступно по адресу https://data.cityofnewyork.us).

In [2]:
# import modules
import certifi
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import urllib3
from urllib3 import request
from unicodedata import normalize


1.1 Файлы данных `CSV` и `Excel`

Файлы данных могут быть получены откуда угодно: от файлов, хранящихся локально на вашем устройстве, до файловых систем облачного хранилища.

Даже при работе в основном с базами данных или внешними `API` использование физических файлов — отличный способ легко использовать данные с метками времени, что может пригодиться при любых временных проблемах с подключением.

In [3]:
# Загрузите данные с портала открытых данных Правительства Москвы —
# данные о Wi-Fi в кинотеатрах, введя в браузер следующий URL-адрес: https://data.mos.ru/opendata/60789?isDynamic=false.
# Будет загружен CSV-файл *.csv. Сохраните этот файл.
# прочитать данные CSV, используя команду Pandas pd.read_csv() в качестве df DataFrame, затем выберите первые пять строк DataFrame с помощью df.head().

**Задание 1.1.** Зайти на сайт URL-адрес: https://data.mos.ru/opendata/60789?isDynamic=false

**Задание 1.2.** Загрузить данные о `Wi-Fi в кинотеатрах` в Colab.

**Задание 1.3.** Считайте данные из файла CSV. Использовать метод `pandas read_csv` для простоты и скорости.

In [ ]:
df_csv = pd.read_csv("data-60789-2024-01-24.csv", delimiter=';')
df_csv.head()

,global_id,ID,CinemaName,AdmArea,District,Address,NumberOfAccessPoints,WiFiName,CoverageArea,FunctionFlag,AccessFlag,Password,Latitude_WGS84,Longitude_WGS84,geoData,geodata_center,Unnamed: 16
0,global_id,Код,Наименование кинотеатра,Административный округ,Район,Адрес,Количество точек доступа,Имя Wi-Fi сети,"Зона покрытия, в метрах",Признак функционирования,Условия доступа,Пароль,Широта в WGS-84,Долгота в WGS-84,geoData,geodata_center,NaN
1,879791944,1,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,Останкинский район,"Российская Федерация, город Москва, внутригоро...",7,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.818675,37.636778,"{coordinates=[37.636778, 55.818675], type=Point}",NaN,NaN
2,879791951,3,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Печатники,"город Москва, улица Кухмистерова, дом 4",8,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.685873,37.718534,"{coordinates=[37.718534, 55.685873], type=Point}",NaN,NaN
3,879791953,4,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Текстильщики,"Российская Федерация, город Москва, внутригоро...",6,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.710864,37.731299,"{coordinates=[37.731299, 55.710864], type=Point}",NaN,NaN
4,879791961,7,Государственное бюджетное учреждение культуры ...,Северный административный округ,Тимирязевский район,"Российская Федерация, город Москва, внутригоро...",3,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.814406,37.571210,"{coordinates=[37.57121, 55.814406], type=Point}",NaN,NaN


**Задание 1.4.** Так как первая строка включает определение столбцов, удалить строку с `index=0`.

In [ ]:
data = df_csv.drop(df_csv.index[0])

In [ ]:
data

,global_id,ID,CinemaName,AdmArea,District,Address,NumberOfAccessPoints,WiFiName,CoverageArea,FunctionFlag,AccessFlag,Password,Latitude_WGS84,Longitude_WGS84,geoData,geodata_center,Unnamed: 16
1,879791944,1,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,Останкинский район,"Российская Федерация, город Москва, внутригоро...",7,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.818675,37.636778,"{coordinates=[37.636778, 55.818675], type=Point}",NaN,NaN
2,879791951,3,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Печатники,"город Москва, улица Кухмистерова, дом 4",8,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.685873,37.718534,"{coordinates=[37.718534, 55.685873], type=Point}",NaN,NaN
3,879791953,4,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Текстильщики,"Российская Федерация, город Москва, внутригоро...",6,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.710864,37.731299,"{coordinates=[37.731299, 55.710864], type=Point}",NaN,NaN
4,879791961,7,Государственное бюджетное учреждение культуры ...,Северный административный округ,Тимирязевский район,"Российская Федерация, город Москва, внутригоро...",3,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.814406,37.571210,"{coordinates=[37.57121, 55.814406], type=Point}",NaN,NaN
5,879791967,9,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,район Свиблово,"Российская Федерация, город Москва, внутригоро...",5,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.851945,37.647549,"{coordinates=[37.647549, 55.851945], type=Point}",NaN,NaN
6,879795206,2,Государственное бюджетное учреждение культуры ...,Северо-Восточный административный округ,Бабушкинский район,"Российская Федерация, город Москва, внутригоро...",5,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.861974,37.676825,"{coordinates=[37.676825, 55.861974], type=Point}",NaN,NaN
7,879795207,5,Государственное бюджетное учреждение культуры ...,Юго-Восточный административный округ,район Лефортово,"Российская Федерация, город Москва, внутригоро...",4,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.764303,37.705508,"{coordinates=[37.705508, 55.764303], type=Point}",NaN,NaN
8,879795208,6,Государственное бюджетное учреждение культуры ...,Юго-Западный административный округ,Академический район,"Российская Федерация, город Москва, внутригоро...",3,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.682787,37.571585,"{coordinates=[37.571585, 55.682787], type=Point}",NaN,NaN
9,879795209,8,Государственное бюджетное учреждение культуры ...,Восточный административный округ,район Новогиреево,"Российская Федерация, город Москва, внутригоро...",6,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.749841,37.802562,"{coordinates=[37.802562, 55.749841], type=Point}",NaN,NaN
10,879795211,10,Государственное бюджетное учреждение культуры ...,Юго-Западный административный округ,Обручевский район,"Российская Федерация, город Москва, внутригоро...",5,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,55.665309,37.513395,"{coordinates=[37.513395, 55.665309], type=Point}",NaN,NaN


# **Задание 1.5.** Выполнить парсинг веб-страницы с помощью `JSON`

In [ ]:
# Импорт стандартных пакетов
import pandas as pd

# Для парсинга веб-страниц
import requests as reqs
import urllib.request
from bs4 import BeautifulSoup
import json
import os
import pprint

# Для выполнения операций с регулярными выражениями
import re

In [ ]:
resp = reqs.request(method='GET', url='https://apidata.mos.ru/v1/datasets/60789/features?api_key=efb65918-60a2-4370-97b3-6f3b4798aa44')
json_page = resp.json()
print(json_page)

{'features': [{'geometry': {'coordinates': [37.636778, 55.818675], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 1, 'CinemaName': 'Государственное бюджетное учреждение культуры города Москвы «Московское кино» Кинотеатр «Москино Космос»', 'AdmArea': 'Северо-Восточный административный округ', 'District': 'Останкинский район', 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Останкинский, проспект Мира, дом 109', 'NumberOfAccessPoints': 7, 'WiFiName': 'Moscow_WiFi_Free', 'CoverageArea': 50, 'FunctionFlag': 'действует', 'AccessFlag': 'открытая сеть', 'Password': '', 'Latitude_WGS84': '55.818675', 'Longitude_WGS84': '37.636778', 'global_id': 879791944}, 'rowId': None, 'versionNumber': 1, 'releaseNumber': 32}, 'type': 'Feature'}, {'geometry': {'coordinates': [37.718534, 55.685873], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 3, 'CinemaName': 'Государственн

## <font color='blue'>Парсинг веб-страниц с помощью JSON</font>

### <font color="green"> Что такое JSON?</font>

* JSON (нотация объектов JavaScript) — популярный формат данных, используемый для представления структурированных данных.
* Это текстовый формат, который не зависит от языка и может использоваться в Python, Perl и других языках.
* Формат JSON используется для передачи данных между серверами и веб-приложениями.
* Он построен на двух структурах:

      - Коллекция пар имя/значение. Это реализуется как объект, запись, словарь, хеш-таблица, список ключей или ассоциативный массив.
      - Упорядоченный список значений. Это реализуется как массив, вектор, список или последовательность.
     
     
Основные функции `JSON`:

* `dump()`: запись закодированной строки в файл.
* `load()`: декодирование во время чтения файла JSON.
* `dumps()`: кодирование в объекты JSON.
* `loads()`: декодирует строку JSON.

### <font color="green"> Сериализация и десериализация</font>

> … процесс перевода структур данных или состояния объекта в формат, который можно сохранить… или передать… и позже восстановить (возможно, в другой компьютерной среде). (Википедия)

* **Сериализация** — это процесс преобразования объектов или структур данных в потоки байтов или строки.
* Эти потоки байтов затем можно легко сохранить или передать.
* Это позволяет разработчикам сохранять, например, данные конфигурации или прогресс пользователя, а затем сохранять их (на диске или в базе данных) или отправлять в другое место.
* Обратный процесс сериализации известен как **десериализация**.

### Зачем нужна сериализация?

Нам нужна сериализация по следующим причинам:

- **Связь**: Сериализация включает в себя процедуру сериализации и передачи объекта. Это позволяет нескольким компьютерным системам одновременно проектировать, совместно использовать и выполнять объекты.
- **Кэширование**: время, затрачиваемое на создание объекта, больше времени, необходимого для его десериализации. Сериализация минимизирует затраты времени за счет кэширования гигантских объектов.
- **Глубокое копирование**: процесс клонирования упрощается благодаря сериализации. Точная копия объекта получается путем сериализации объекта в массив байтов и последующей его десериализации.
- **Переносимость**. Основное преимущество сериализации заключается в том, что она работает в разных архитектурах и операционных системах.
- **Постоянство**: состояние любого объекта можно сохранить напрямую, применив к нему сериализацию, и сохранить в базе данных, чтобы его можно было получить позже.

**Сериализация**

Мы используем функцию dump(), которая принимает два аргумента:
* Объект данных, подлежащий сериализации.
* Объект файла, в который он будет записан (байтовый формат).

In [ ]:
file_name = "Sample.json"
with open(file_name, "w") as fid:
     json.dump(json_page, fid)

In [ ]:
!cat Sample.json

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


**Десериализация JSON**

* Десериализация противоположна сериализации, т. е. преобразованию объекта JSON в соответствующие объекты Python.
* Мы используем функцию load(), которая обычно используется для загрузки из строки, в противном случае корневой объект находится в списке или диктовке.

In [ ]:
with open(file_name, "r") as fid:
     json_obj = json.load(fid)

print(json_obj)

{'features': [{'geometry': {'coordinates': [37.636778, 55.818675], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 1, 'CinemaName': 'Государственное бюджетное учреждение культуры города Москвы «Московское кино» Кинотеатр «Москино Космос»', 'AdmArea': 'Северо-Восточный административный округ', 'District': 'Останкинский район', 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Останкинский, проспект Мира, дом 109', 'NumberOfAccessPoints': 7, 'WiFiName': 'Moscow_WiFi_Free', 'CoverageArea': 50, 'FunctionFlag': 'действует', 'AccessFlag': 'открытая сеть', 'Password': '', 'Latitude_WGS84': '55.818675', 'Longitude_WGS84': '37.636778', 'global_id': 879791944}, 'rowId': None, 'versionNumber': 1, 'releaseNumber': 32}, 'type': 'Feature'}, {'geometry': {'coordinates': [37.718534, 55.685873], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 3, 'CinemaName': 'Государственн

In [ ]:
for key in json_obj:
    print("{:>12}: {}".format(key, json_obj[key]))

    features: [{'geometry': {'coordinates': [37.636778, 55.818675], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 1, 'CinemaName': 'Государственное бюджетное учреждение культуры города Москвы «Московское кино» Кинотеатр «Москино Космос»', 'AdmArea': 'Северо-Восточный административный округ', 'District': 'Останкинский район', 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Останкинский, проспект Мира, дом 109', 'NumberOfAccessPoints': 7, 'WiFiName': 'Moscow_WiFi_Free', 'CoverageArea': 50, 'FunctionFlag': 'действует', 'AccessFlag': 'открытая сеть', 'Password': '', 'Latitude_WGS84': '55.818675', 'Longitude_WGS84': '37.636778', 'global_id': 879791944}, 'rowId': None, 'versionNumber': 1, 'releaseNumber': 32}, 'type': 'Feature'}, {'geometry': {'coordinates': [37.718534, 55.685873], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 3, 'CinemaName': 'Государствен

Обрабатываем данные с помощью `JSON`:

In [ ]:
import requests

In [ ]:
api_url=('https://apidata.mos.ru/v1/datasets/60789/features?api_key=efb65918-60a2-4370-97b3-6f3b4798aa44')

In [ ]:
response = requests.get(api_url)

In [ ]:
if response.status_code == 200:
   json_page = json.loads(response.text)

**Задание 1.6.** Выполнить парсинг веб-страницы с помощью `JSON`. Получите URL-адрес:

In [ ]:
print(f"URL: \n\t {response.url}")

URL: 
	 https://apidata.mos.ru/v1/datasets/60789/features?api_key=efb65918-60a2-4370-97b3-6f3b4798aa44


**Задание 1.7.** Выполнить парсинг веб-страницы с помощью `JSON`. Определите тип контента:

In [ ]:
print(f"Content type: \n\t {response.headers['content-type']}")

Content type: 
	 application/json


**Задание 1.8.** Выполнить парсинг веб-страницы с помощью `JSON`. Обработайте данные с помощью JSON:

In [ ]:
if response.status_code == 200:
   json_page = json.loads(response.text)

**Задание 1.9.**  Обработайте данные с помощью `JSON`. Посмотреть на ключи:

In [ ]:
for x in json_page:
    print(x)

features
type


Распечатать ключи и значения:

In [ ]:
for x in json_page:
    print(f"{x} --> {json_page[x]} \n")

features --> [{'geometry': {'coordinates': [37.636778, 55.818675], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 1, 'CinemaName': 'Государственное бюджетное учреждение культуры города Москвы «Московское кино» Кинотеатр «Москино Космос»', 'AdmArea': 'Северо-Восточный административный округ', 'District': 'Останкинский район', 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Останкинский, проспект Мира, дом 109', 'NumberOfAccessPoints': 7, 'WiFiName': 'Moscow_WiFi_Free', 'CoverageArea': 50, 'FunctionFlag': 'действует', 'AccessFlag': 'открытая сеть', 'Password': '', 'Latitude_WGS84': '55.818675', 'Longitude_WGS84': '37.636778', 'global_id': 879791944}, 'rowId': None, 'versionNumber': 1, 'releaseNumber': 32}, 'type': 'Feature'}, {'geometry': {'coordinates': [37.718534, 55.685873], 'type': 'Point'}, 'properties': {'datasetId': 60789, 'attributes': {'is_deleted': 0, 'ID': 3, 'CinemaName': 'Государственн

In [ ]:
pprint.pprint(json_page)

{'features': [{'geometry': {'coordinates': [37.636778, 55.818675],
                            'type': 'Point'},
               'properties': {'attributes': {'AccessFlag': 'открытая сеть',
                                             'Address': 'Российская Федерация, '
                                                        'город Москва, '
                                                        'внутригородская '
                                                        'территория '
                                                        'муниципальный округ '
                                                        'Останкинский, '
                                                        'проспект Мира, дом '
                                                        '109',
                                             'AdmArea': 'Северо-Восточный '
                                                        'административный '
                                                        'округ',
  

# **Задание 1.10.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`

 <font color='blue'>Парсинг веб-страниц с помощью Beautiful Soup</font>

- Веб-скрапинг позволяет загрузить HTML-код веб-сайта и извлечь необходимые вам данные.
- Beautiful Soup — это библиотека Python для очистки данных с веб-сайтов.
- Beautiful Soup создает дерево разбора из проанализированных документов HTML и XML.

In [ ]:
from bs4 import BeautifulSoup as bso

In [ ]:
# Отправление запроса на получение данных и присваивание ответ переменной.
source = reqs.get("https://apidata.mos.ru/v1/datasets/60789/features?api_key=efb65918-60a2-4370-97b3-6f3b4798aa44")

In [ ]:
print(f"Content type: \n\t {source.headers['content-type']}")

Content type: 
	 application/json


In [ ]:
print(source)

<Response [200]>


**Задание 1.10.1.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Создать beautiful soup объект.

In [ ]:
mysoup = bso(source.text, 'html.parser')

**Задание 1.10.2.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Распечатайте HTML-содержимое страницы, используя метод prettify

In [ ]:
print(mysoup.prettify())

{"features":[{"geometry":{"coordinates":[37.636778,55.818675],"type":"Point"},"properties":{"datasetId":60789,"attributes":{"is_deleted":0,"ID":1,"CinemaName":"Государственное бюджетное учреждение культуры города Москвы «Московское кино» Кинотеатр «Москино Космос»","AdmArea":"Северо-Восточный административный округ","District":"Останкинский район","Address":"Российская Федерация, город Москва, внутригородская территория муниципальный округ Останкинский, проспект Мира, дом 109","NumberOfAccessPoints":7,"WiFiName":"Moscow_WiFi_Free","CoverageArea":50,"FunctionFlag":"действует","AccessFlag":"открытая сеть","Password":"","Latitude_WGS84":"55.818675","Longitude_WGS84":"37.636778","global_id":879791944},"rowId":null,"versionNumber":1,"releaseNumber":32},"type":"Feature"},{"geometry":{"coordinates":[37.718534,55.685873],"type":"Point"},"properties":{"datasetId":60789,"attributes":{"is_deleted":0,"ID":3,"CinemaName":"Государственное бюджетное учреждение культуры города Москвы «Московское кино»

**Задание 1.10.3.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Получить заголовок страницы

In [ ]:
print(mysoup.title)

None


# Получение данных из таблиц РСУБД

**Задание 1.11.** Считать результаты запроса sqlite в DataFrame pandas

In [ ]:
# Считываем результаты запроса sqlite в DataFrame pandas
with sqlite3.connect("movies.sqlite") as conn:
    df = pd.read_sql("SELECT * from movies", conn)
df.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


# Получение данных `Parquet`

Посетите URL https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

In [ ]:

pip install fastparquet


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/670.5 kB ? eta -:--:--
     ------                                 112.6/670.5 kB 2.2 MB/s eta 0:00:01
     ----------------                       286.7/670.5 kB 2.9 MB/s eta 0:00:01
     --------------------------------       573.4/670.5 kB 4.0 MB/s eta 0:00:01
     -------------------------------------- 670.5/670.5 kB 3.8 MB/s eta 0:00:00
                                              0.0/1.6 MB ? eta -:--:--
     -------                                  0.3/1.6 MB 6.7 MB/s eta 0:00:01
     ---------------                          0.6/1.6 MB 6.7 MB/s eta 0:00:01
     -----------------------                  0.9/1.6 MB 6.5 MB/s eta 0:00:01
     ------------------------------           1.2/1.6 MB 7.0 MB/s eta 0:00:01
     -------------------------------------    1.5/1.6 MB 6.8 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.6 MB 6.8 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 5.7 MB/s e

In [ ]:
pd.read_parquet("yellow_tripdata_2023-03.parquet")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2.0,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238.0,42.0,2.0,8.60,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00
1,2.0,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138.0,231.0,1.0,52.70,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25
2,1.0,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140.0,186.0,1.0,18.40,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00
3,1.0,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140.0,43.0,1.0,15.60,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00
4,2.0,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79.0,137.0,1.0,7.20,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3403761,2.0,2023-03-31 23:24:25,2023-03-31 23:40:54,NaN,3.16,NaN,None,163.0,75.0,0.0,12.13,0.0,0.5,4.23,0.0,1.0,20.36,NaN,NaN
3403762,2.0,2023-03-31 23:24:50,2023-04-01 00:04:12,NaN,6.89,NaN,None,125.0,198.0,0.0,40.92,0.0,0.5,8.98,0.0,1.0,53.90,NaN,NaN
3403763,2.0,2023-03-31 23:26:31,2023-03-31 23:49:39,NaN,4.01,NaN,None,50.0,224.0,0.0,24.02,0.0,0.5,0.00,0.0,1.0,28.02,NaN,NaN
3403764,2.0,2023-03-31 23:07:51,2023-03-31 23:15:56,NaN,1.31,NaN,None,113.0,158.0,0.0,8.51,0.0,0.5,3.50,0.0,1.0,16.01,NaN,NaN


**Задание 1.12.** Считайте данные из файла Parquet. Использовать метод pandas read_parquet.

In [ ]:
df_parquet.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2.0,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238.0,42.0,2.0,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00
1,2.0,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138.0,231.0,1.0,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25
2,1.0,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140.0,186.0,1.0,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00
3,1.0,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140.0,43.0,1.0,15.6,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00
4,2.0,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79.0,137.0,1.0,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00
